In [15]:
from openmm import *
from openmm.app import *
from openmm.unit import *
from sys import stdout

In [3]:
pdbfile = PDBFile('data/prep_complex.pdb')

modeller = Modeller(pdbfile.topology, pdbfile.positions)

forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")
modeller.addSolvent(forcefield, model="tip3p", padding=0.5 * nanometer, boxShape='dodecahedron')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=500)
positions = simulation.context.getState(getPositions=True).getPositions()
with open("init_romb.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, positions, f)

In [12]:
#Or load existing minimized system
pdbfile = PDBFile("init_romb.pdb")

modeller = Modeller(pdbfile.topology, pdbfile.positions)

forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

In [13]:
simulation.reporters = []
simulation.reporters.append(DCDReporter("traj.dcd", 500))
simulation.reporters.append(
    StateDataReporter(stdout, 100, step=True, temperature=True, elapsedTime=True)
)
simulation.reporters.append(
    StateDataReporter(
        "scalars.csv",
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
        density=True
    )
)
simulation.step(5000)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

#"Step","Temperature (K)","Elapsed Time (s)"
100,80.1478306968461,0.00011515617370605469
200,126.21692376759508,0.3947322368621826
300,160.54763433428215,0.4982268810272217
400,187.99807831683708,0.5935156345367432
500,209.8015408893179,0.6992671489715576
600,224.21705389615744,0.7609167098999023
700,239.30012464039626,0.8344826698303223
800,246.64701905975028,0.908574104309082
900,256.8042793279641,0.9775633811950684
1000,264.4254311111532,1.0784900188446045
1100,268.66362674568836,1.1424074172973633
1200,278.8372781452872,1.2149081230163574
1300,277.33428920809945,1.2871642112731934
1400,286.98882200706794,1.357492446899414
1500,288.70212531238286,1.4572157859802246
1600,288.7586180963081,1.5214512348175049
1700,288.775177112178,1.5943093299865723
1800,293.11065823074455,1.667771816253662
1900,291.0684612011641,1.737682580947876
2000,296.1632715419431,1.8374097347259521
2100,292.84543869954683,1.9014759063720703
2200,296.6728483401855,1.9745101928710938
2300,294.82701728705575,2.0492

In [17]:
pdbfile = PDBFile('data/prep_complex.pdb')

modeller = Modeller(pdbfile.topology, pdbfile.positions)

forcefield = ForceField("amber14-all.xml", "implicit/gbn2.xml")
system = forcefield.createSystem(modeller.topology, nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1.5*nanometer, constraints=HBonds,
                                 soluteDielectric=1.0, solventDielectric=80.0, implicitSolventKappa=1.0/nanometer)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=500)
positions = simulation.context.getState(getPositions=True).getPositions()
with open("init_impl.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, positions, f)

In [5]:
simulation.reporters = []
simulation.reporters.append(DCDReporter("traj.dcd", 100))
simulation.reporters.append(
    StateDataReporter(stdout, 100, step=True, temperature=True, elapsedTime=True)
)
simulation.reporters.append(
    StateDataReporter(
        "scalars.csv",
        10,
        step=True,
        time=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True
    )
)
simulation.step(3000)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

#"Step","Temperature (K)","Elapsed Time (s)"
100,57.89275132149787,8.7738037109375e-05
200,102.60063026168775,0.06796932220458984
300,137.0945776674555,0.13356685638427734
400,163.52537252884787,0.20147705078125
500,183.83788724824524,0.2392117977142334
600,209.0948894676991,0.27362513542175293
700,225.45974432120806,0.30774807929992676
800,249.10062321073508,0.3414449691772461
900,253.6409700936284,0.3752784729003906
1000,254.6871507680923,0.40927624702453613
1100,271.61561678076436,0.44086790084838867
1200,269.3382913452907,0.4717226028442383
1300,271.1539974807957,0.5029873847961426
1400,280.56861638386385,0.533836841583252
1500,286.5074548338407,0.5648174285888672
1600,278.6584128357336,0.5957331657409668
1700,285.35357687863325,0.6273651123046875
1800,282.81099544900076,0.6586740016937256
1900,293.5804064777359,0.6898400783538818
2000,292.49255989288463,0.7207162380218506
2100,287.83445963593516,0.7516703605651855
2200,288.10969050089716,0.7828278541564941
2300,298.4974688150854,0